In [2]:
import pandas as pd
import numpy as np

### The `where()` method and masking ###

从Series中利用boolean vector选择Series, 会返回data object的子集. 当需要返回的数据与原数据具有相同的shape时, 可以在Series对象上使用 `where` 方法.

In [3]:
s1 = pd.Series(np.arange(5), index=np.arange(5)[::-1], dtype='int64')
s1

4    0
3    1
2    2
1    3
0    4
dtype: int64

1. 使用 boolean vector选择符合条件的值, 返回值shape与原数据对象不同
2. 使用where选择符合条件的值, 返回值shape与原数据对象相同

In [5]:
s1[s1 > 0]

3    1
2    2
1    3
0    4
dtype: int64

In [6]:
s1.where(s1 > 0)

4    NaN
3    1.0
2    2.0
1    3.0
0    4.0
dtype: float64

如果在DataFrame对象上使用boolean条件进行索引, 会得到与原DataFrame的shape相同的返回值. Pandas使用`where`函数实现这一类索引

In [7]:
df = pd.DataFrame({'vals': [1, 2, 3, 4], 'ids': ['a', 'b', 'f', 'n'],'ids2': ['a', 'n', 'c', 'n']})
df

,ids,ids2,vals
0,a,a,1
1,b,n,2
2,f,c,3
3,n,n,4


`df[df < 2]` 的效果与 `df.where(df < 2)` 效果相同

In [21]:
df[df < 2]

,ids,ids2,vals
0,a,a,1.0
1,b,n,NaN
2,f,c,NaN
3,n,n,NaN


In [22]:
df.where(df < 2)

,ids,ids2,vals
0,a,a,1.0
1,b,n,NaN
2,f,c,NaN
3,n,n,NaN


另外, `where()` 方法接收一个可选的 `other` 参数, 用于替换不满足条件的值.

In [25]:
df.where(df < 2, 0)

,ids,ids2,vals
0,a,a,1
1,b,n,0
2,f,c,0
3,n,n,0


In [35]:
df2 = pd.DataFrame(np.random.randn(4,5), columns=list('abcde'))
df2

,a,b,c,d,e
0,0.134353,-1.968575,0.224103,-0.440970,1.176074
1,2.064340,-1.618674,-0.576543,-1.357081,0.776904
2,0.194596,0.035384,0.912971,-0.372574,0.280662
3,0.039102,-2.478352,-0.990907,-1.356812,-0.310479


In [60]:
# 将df2中所有小于0的值都 +1, 可以在other 参数中传入lambda表达式. lambda 表达式有一个参数(Series 或者DataFrame)
df2.where(df2 > 0, lambda df : df + 1)

,a,b,c,d,e
0,0.134353,-0.968575,0.224103,0.559030,1.176074
1,2.064340,-0.618674,0.423457,-0.357081,0.776904
2,0.194596,0.035384,0.912971,0.627426,0.280662
3,0.039102,-1.478352,0.009093,-0.356812,0.689521


#### alignment ####

`where` 可以对齐输入的的boolean条件(ndarray 或者 DataFrame), 使得 partial selection并赋值是可能的

In [38]:
df3 = df2.copy()

In [39]:
df3

,a,b,c,d,e
0,0.134353,-1.968575,0.224103,-0.440970,1.176074
1,2.064340,-1.618674,-0.576543,-1.357081,0.776904
2,0.194596,0.035384,0.912971,-0.372574,0.280662
3,0.039102,-2.478352,-0.990907,-1.356812,-0.310479


In [46]:
df3[1:3] > 0

,a,b,c,d,e
1,True,False,False,False,True
2,True,True,True,False,True


In [47]:
# 选择df3的1，2行中大于0的元素. 此时Pandas会对齐原DataFrame的Index与boolean DataFrame的Index
df3[df3[1:3] > 0]

,a,b,c,d,e
0,NaN,NaN,NaN,NaN,NaN
1,2.064340,NaN,NaN,NaN,0.776904
2,0.194596,0.035384,0.912971,NaN,0.280662
3,NaN,NaN,NaN,NaN,NaN


In [48]:
# 在以上的基础上, 对选出的element赋值为3
df3[df3[1:3] > 0] = 3

In [49]:
df3

,a,b,c,d,e
0,0.134353,-1.968575,0.224103,-0.440970,1.176074
1,3.000000,-1.618674,-0.576543,-1.357081,3.000000
2,3.000000,3.000000,3.000000,-0.372574,3.000000
3,0.039102,-2.478352,-0.990907,-1.356812,-0.310479


我们也可以使用 `where` 方法提供的axis参数和level参数去选择对齐的方式

In [51]:
df4 = df2.copy()
df4

,a,b,c,d,e
0,0.134353,-1.968575,0.224103,-0.440970,1.176074
1,2.064340,-1.618674,-0.576543,-1.357081,0.776904
2,0.194596,0.035384,0.912971,-0.372574,0.280662
3,0.039102,-2.478352,-0.990907,-1.356812,-0.310479


In [54]:
# 选择出a列, 1-3行的数据
df4.loc[1:3, 'a']

1    2.064340
2    0.194596
3    0.039102
Name: a, dtype: float64

In [55]:
# 构建df4 > 0的条件
df4 > 0

,a,b,c,d,e
0,True,False,True,False,True
1,True,False,False,False,True
2,True,True,True,False,True
3,True,False,False,False,False


In [56]:
# 对于不符合 df4>0 条件的element, 将其赋值为 df4.loc[1:3, 'a'] 中对应的值, 使用 axis = 'index' 定义对应方式为index相同
df4.where(df4 > 0, df4.loc[1:3, 'a'], axis='index')

,a,b,c,d,e
0,0.134353,NaN,0.224103,NaN,1.176074
1,2.064340,2.064340,2.064340,2.064340,0.776904
2,0.194596,0.035384,0.912971,0.194596,0.280662
3,0.039102,0.039102,0.039102,0.039102,0.039102


执行以上代码后, 可以发现, (0, b), (0, d)的值为 NaN. 这两个element不符合 `>0` 的条件, 同时 `df4.loc[1:3, 'a']` 中没有对应的index = 0 的值. 因此这两个元素的值为NaN. 以上的代码效果与以下代码等价, 但是速度更快.

In [58]:
df5 = df2.copy()
df5

,a,b,c,d,e
0,0.134353,-1.968575,0.224103,-0.440970,1.176074
1,2.064340,-1.618674,-0.576543,-1.357081,0.776904
2,0.194596,0.035384,0.912971,-0.372574,0.280662
3,0.039102,-2.478352,-0.990907,-1.356812,-0.310479


In [59]:
df5.apply(lambda x, y: x.where(x > 0, y), y = df5.loc[1:3, 'a'])

,a,b,c,d,e
0,0.134353,NaN,0.224103,NaN,1.176074
1,2.064340,2.064340,2.064340,2.064340,0.776904
2,0.194596,0.035384,0.912971,0.194596,0.280662
3,0.039102,0.039102,0.039102,0.039102,0.039102


#### callable as input of where() method ####

上面简单提到了使用 lambda 表达式作为where 方法的other argument. where方法可以接收lambda或者其它callable参数作为condition 和 other 参数. callable 接收一个参数, 这个参数是Series或者DataFrame

In [61]:
df6 = pd.DataFrame({'A': [1, 2, 3],'B': [4, 5, 6],'C': [7, 8, 9]})
df6

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [62]:
# 对df6中的所有小于4的数, 都 +10
df6.where(lambda df : df > 4, lambda df : df + 10)

,A,B,C
0,11,14,7
1,12,5,8
2,13,6,9


事实上, `where()` 方法对